this script was created per Rich's request. Goal is to calculate total volume of water withdrawal in 2014


In [16]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2017M11D14', 'UTC 11:43')


'2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [1]:
import ee
import pandas as pd
import os

In [2]:
ee.Initialize()

In [3]:
SCRIPT_NAME = "Y2017M11D14_RH_Rich_PCR_Sums_V01"
OUTPUT_VERSION =1 

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/%s/output/" %(SCRIPT_NAME)

OUTPUT_FILE_NAME = "Y2017M11D14_RH_Rich_PCR_Sums_V%0.2d" %(OUTPUT_VERSION)

In [4]:
!rm -r {EC2_OUTPUT_PATH}

!mkdir -p {EC2_OUTPUT_PATH}

In [5]:
year = 2014
timeFrame = "year" #annual


In [6]:
sectors = ["Dom","Ind","Irr","Liv"]

In [7]:
demandTypes = ["WW","WN"]

In [8]:
geometry = ee.Geometry.Polygon(coords=[[-180.0, -90.0], [180,  -90.0], [180, 90], [-180,90]], proj= ee.Projection('EPSG:4326'),geodesic=False )

In [9]:
scale = ee.Image(ee.ImageCollection("projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWN_year_millionm3_5min_1960_2014").first()).projection().nominalScale().getInfo()

In [10]:
print(scale)

9276.62397058


In [11]:
df = pd.DataFrame()
fc = ee.FeatureCollection(ee.Feature(None,{}))
for year in range(1960,2015):
    
    print(year)
    for sector in sectors:
        for demandType in demandTypes:
            indicator = "%s%s" %(sector,demandType)
            keyName = "%s%sY%0.2d" %(sector,demandType,year)
            filePath = "projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_P%s_year_millionm3_5min_1960_2014" %(indicator)
            ic = ee.ImageCollection(filePath)
            imageYear = ee.Image(ic.filter(ee.Filter.eq("year",year)).first())
            sumYear = imageYear.reduceRegion(
                geometry= geometry,
                reducer= ee.Reducer.sum(),
                scale= scale,
                maxPixels= 1e10
            )
            df.at[year, indicator] = sumYear.get("b1").getInfo()
            
               
    

1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


In [12]:
df

,DomWW,DomWN,IndWW,IndWN,IrrWW,IrrWN,LivWW,LivWN
1960,82829.934120,53617.348704,3.501228e+05,79004.246007,1.487135e+06,5.496761e+05,10591.382454,10591.382454
1961,84804.556401,54441.266194,3.514145e+05,79179.859626,1.460775e+06,5.441747e+05,10679.352846,10679.352846
1962,87361.339997,56054.475729,3.561983e+05,82715.303385,1.514793e+06,5.615826e+05,10796.260742,10796.260742
1963,90545.782340,58139.043652,3.618173e+05,84065.468090,1.551366e+06,5.730279e+05,10993.763545,10993.763545
1964,94530.612240,60370.855740,3.705921e+05,83113.262957,1.508349e+06,5.614052e+05,11259.557907,11259.557907
1965,98348.335283,61208.121595,3.785024e+05,84914.371506,1.638235e+06,6.083811e+05,11423.882174,11423.882174
1966,101295.826568,62448.283176,3.852550e+05,86244.782128,1.714470e+06,6.377648e+05,11590.636581,11590.636581
1967,103977.924693,63664.144054,3.901469e+05,88020.731657,1.636986e+06,6.094680e+05,11812.188414,11812.188414
1968,107521.942523,65407.297339,3.981396e+05,89821.162168,1.704831e+06,6.322132e+05,11978.003904,11978.003904
1969,112210.936529,67940.712904,4.086698e+05,92321.764772,1.738357e+06,6.418879e+05,12016.465769,12016.465769


In [14]:
df.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".csv"))

In [15]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../data/Y2017M11D14_RH_Rich_PCR_Sums_V01/output/Y2017M11D14_RH_Rich_PCR_Sums_V01 to s3://wri-projects/Aqueduct30/processData/Y2017M11D14_RH_Rich_PCR_Sums_V01/output/Y2017M11D14_RH_Rich_PCR_Sums_V01


In [17]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:00:03.418900
